# Installation des libairies

In [1]:
#!pip install findspark
#!pip install pyspark
#!pip install streamlit
#!pip install nympy
#!pip install plotly

# Import des librairies

In [2]:
import findspark
findspark.init()
findspark.find()

'/Users/pierrick/opt/miniconda3/lib/python3.9/site-packages/pyspark'

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as spf
import pandas as pd
import numpy as np
#import streamlit as st
import plotly as plt

# Traitement ETL
## Création d'un SparkSession

In [4]:
spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/06 16:54:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalAccessError: class org.apache.spark.storage.StorageUtils$ (in unnamed module @0x3c60b7e7) cannot access class sun.nio.ch.DirectBuffer (in module java.base) because module java.base does not export sun.nio.ch to unnamed module @0x3c60b7e7
	at org.apache.spark.storage.StorageUtils$.<init>(StorageUtils.scala:213)
	at org.apache.spark.storage.StorageUtils$.<clinit>(StorageUtils.scala)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.<init>(BlockManagerMasterEndpoint.scala:110)
	at org.apache.spark.SparkEnv$.$anonfun$create$9(SparkEnv.scala:348)
	at org.apache.spark.SparkEnv$.registerOrLookupEndpoint$1(SparkEnv.scala:287)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:336)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:191)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:277)
	at org.apache.spark.SparkContext.<init>(SparkContext.s

## Fonction d'ouverture de fichier

In [ ]:
def openDataFrame(data):
    df = spark.read.option("delimiter", ",") \
        .csv(f"data/raw-file/{data}.csv", header=True)
    df.printSchema()
    return df

## Liste des fichiers à traiter

In [ ]:
fileName = ["dre_auvergne_rhone_alpes",
            "dre_bourgogne_franche_comte",
            "dre_bretagne",
            "dre_centre_val_de_loire",
            "dre_corse",
            "dre_france_metropolitaine",
            "dre_grand_est",
            "dre_hauts_de_france",
            "dre_ile_de_france",
            "dre_normandie",
            "dre_nouvelle_aquitaine",
            "dre_occitanie",
            "dre_pays_de_la_loire",
            "dre_provence_alpes_cote_d_azur",
           ]

### Enlève la limitation de l'affichage du dataframe

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

## Ouverture d'un fichier .csv et affichage de dataframe

In [ ]:
df = openDataFrame(fileName[0])
df

## Transposition de l'affichage

In [ ]:
df.show(3, vertical=True)

Affichage des colonnes

In [ ]:
df.columns

Affichage des données

In [ ]:
df.printSchema()

La deuxième colonne ne porte pas de nom ..
Nous allons y remèdier maintenant 😉

In [ ]:
# def withColumnRenamed(existingName: String, newName: String): DataFrame
df = df.withColumnRenamed("_c1","Intitulé énergie")
df

Sommaire des données

In [ ]:
df.select('2014', '2015').describe().show()

In [ ]:
# Permière version des liste

list_production = [
    'P1','P2','P3','P4','P5','P7','P8','P9','P10','P11','P12','P13','P14',
    'E1','E2','E4','E6','E7','E8','E9','E10','E11','E12','E13','E14','E15','E16','E17','E18','E19','E20','E21','E22','E23','E24','E25','E26','E27','E28'
]

list_consomation = [
    'C1','CI1','CI3','CI3b','CI4','CI9','CI11','CI5','CI6','CI7','CI8',
    'CTR1','CTR2','CTR3','CTR4','CTR5','CTR6','CTR7','CTR8','CTR9','CTR10','CTR11',
    'CR1','CR7','CR2','CR3','CR4','CR5','CR6','CR8',
    'CTE1','CTE7','CTE2','CTE3','CTE4','CTE5','CTE6','CTE8',
    'CA1','CA7','CA2','CA21','CA3','CA4','CA5','CA6','CA8',
    'C2','C3','C4','C13','C14'
]

# Penser à passer par une tranposition puis pour un passae d'indexe

In [ ]:
df_production = df.filter(df.PRODUCTION.isin(list_production))
df_production.show()

## Liste de test non concluant
#subset_df = df.filter(df.PRODUCTION.isin(list_production))
#df_production = spark.createDataFrame(subset_df, df.columns)
#df_production = df.withColumn(df.columns[0], spf.when(df.PRODUCTION.isin(list_production)))
#df_production.show

In [ ]:
df_consomation = df.filter(df.PRODUCTION.isin(list_consomation))
df_consomation.show()

## Sauvegardes 

###  /!\ En Chantier

In [ ]:
def saveDataFrame(dataframe, file_name):
    file = f"data/refined-file/{file_name}.csv"
    dataframe.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save(file)
    return f"{file_name} as been succesfully create"

In [ ]:
def i(dataframe):
    dataframe.write.format('jdbc').options(
      url='jdbc:mysql://localhost/spark_nrg',
      driver='com.mysql.jdbc.Driver',
      dbtable='D_TEMP',
      user='user',
      password='user').mode('append').save()

In [ ]:
file = f"production_{fileName[0]}"

In [ ]:
test(df_production)

In [ ]:
saveDataFrame(df_production, file)

In [ ]:
saveDataFrame(df_consomation, file)